In [1]:
import cirq

In [2]:
qubits = [cirq.GridQubit(x, y) for x in range(3) for y in range(3)]

In [3]:
print(qubits[0])

(0, 0)


In [5]:
x_gate = cirq.X

x_op = x_gate(qubits[0])

print(x_op)

X((0, 0))


In [6]:
cz = cirq.CZ(qubits[0], qubits[1])
x = cirq.X(qubits[2])
moment = cirq.Moment([x, cz])

print(moment)

X((0, 2)) and CZ((0, 0), (0, 1))


In [7]:
cz01 = cirq.CZ(qubits[0], qubits[1])
x2 = cirq.X(qubits[2])
cz12 = cirq.CZ(qubits[1], qubits[2])
moment0 = cirq.Moment([cz01, x2])
moment1 = cirq.Moment([cz12])

circuit = cirq.Circuit((moment0, moment1))

print(circuit)

(0, 0): ───@───────
           │
(0, 1): ───@───@───
               │
(0, 2): ───X───@───


In [8]:
from cirq.ops import CZ, H

In [10]:
q0, q1, q2 = [cirq.GridQubit(i, 0) for i in range(3)]

In [11]:
circuit = cirq.Circuit()
circuit.append([CZ(q0, q1), H(q2)])

In [12]:
print(circuit)

(0, 0): ───@───
           │
(1, 0): ───@───

(2, 0): ───H───


In [13]:
circuit.append([H(q0), CZ(q1, q2)])

print(circuit)

(0, 0): ───@───H───
           │
(1, 0): ───@───@───
               │
(2, 0): ───H───@───


In [14]:
circuit = cirq.Circuit()
circuit.append([CZ(q0, q1), H(q2), H(q0), CZ(q1, q2)])

print(circuit)

(0, 0): ───@───H───
           │
(1, 0): ───@───@───
               │
(2, 0): ───H───@───


In [15]:
from cirq.circuits import InsertStrategy

In [16]:
circuit = cirq.Circuit()
circuit.append([CZ(q0, q1)])
circuit.append([H(q0), H(q2)], strategy=InsertStrategy.EARLIEST)

print(circuit)

(0, 0): ───@───H───
           │
(1, 0): ───@───────

(2, 0): ───H───────


In [17]:
circuit = cirq.Circuit()
circuit.append([H(q0), H(q1), H(q2)], strategy=InsertStrategy.NEW)

print(circuit)

(0, 0): ───H───────────

(1, 0): ───────H───────

(2, 0): ───────────H───


In [20]:
circuit = cirq.Circuit()
circuit.append([CZ(q1, q2)])
circuit.append([CZ(q1, q2)])
circuit.append([H(q0), H(q1), H(q2)], strategy=InsertStrategy.INLINE)

print(circuit)

(0, 0): ───────H───────

(1, 0): ───@───@───H───
           │   │
(2, 0): ───@───@───H───


In [21]:
circuit = cirq.Circuit()
circuit.append([H(q0)])
circuit.append([CZ(q1, q2), H(q0)], strategy=InsertStrategy.NEW_THEN_INLINE)

print(circuit)

(0, 0): ───H───H───

(1, 0): ───────@───
               │
(2, 0): ───────@───


In [22]:
def my_layer():
    yield CZ(q0, q1)
    yield [H(q) for q in (q0, q1, q2)]
    yield [CZ(q1, q2)]
    yield [H(q0), [CZ(q1, q2)]]

circuit = cirq.Circuit()
circuit.append(my_layer())

for x in my_layer():
    print(x)

CZ((0, 0), (1, 0))
[cirq.H(cirq.GridQubit(0, 0)), cirq.H(cirq.GridQubit(1, 0)), cirq.H(cirq.GridQubit(2, 0))]
[cirq.CZ(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]
[cirq.H(cirq.GridQubit(0, 0)), [cirq.CZ(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]]


In [23]:
print(circuit)

(0, 0): ───@───H───H───────
           │
(1, 0): ───@───H───@───@───
                   │   │
(2, 0): ───H───────@───@───


In [24]:
circuit = cirq.Circuit(H(q0), H(q1))
print(circuit)

(0, 0): ───H───

(1, 0): ───H───


In [25]:
circuit = cirq.Circuit(H(q0), CZ(q0, q1))
for moment in circuit:
    print(moment)

H((0, 0))
CZ((0, 0), (1, 0))


In [27]:
circuit = cirq.Circuit(H(q0), CZ(q0, q1), H(q1), CZ(q0, q1))
print(circuit)
print(circuit[1:3])

(0, 0): ───H───@───────@───
               │       │
(1, 0): ───────@───H───@───
(0, 0): ───@───────
           │
(1, 0): ───@───H───


In [28]:
print(circuit[::-1])

(0, 0): ───@───────@───H───
           │       │
(1, 0): ───@───H───@───────


In [29]:
c = cirq.Circuit()
c.append(cirq.Moment([]))
c.append(cirq.Moment([cirq.X(cirq.GridQubit(1, 1))]))
c.append(cirq.Moment([]))
print('Before optimization, Circuit has {} moments'.format(len(c)))

Before optimization, Circuit has 3 moments


In [30]:
cirq.DropEmptyMoments().optimize_circuit(circuit=c)
print('After optimization, circuit has {} moments'.format(len(c)))

After optimization, circuit has 1 moments


In [31]:
class RemoveMeasurements(cirq.PointOptimizer):
    def optimization_at(self, circuit: cirq.Circuit, index: int, op: cirq.Operation):
        if isinstance(op.gate, cirq.MeasurementGate):
            return cirq.PointOptimizationSummary(clear_span=1, new_operations=[], clear_qubits=op.qubits)
        else:
            return None

In [32]:
q = cirq.LineQubit(0)
c = cirq.Circuit(cirq.Z(q), cirq.measure(q))
print('Before optimization')
print(c)

Before optimization
0: ───Z───M───


In [33]:
RemoveMeasurements().optimize_circuit(c)
print('After optimization')
print(c)

After optimization
0: ───Z───────
